In [1]:
import pandas as pd
import pickle as pkl
import numpy as np

Northwest HVAC Sales Data analysis (BPA TO 43). Starting points for this workbook were outputs from SQL code written by Elizabeth. Sales data is from 2016 thru 2020. Extrapolation is applied to fill temporal gaps in the data. HARDI regional data is used to adjust for geographic data gaps (in Idaho). 

In [2]:
# Starting point/data for this analysis is CSV files generated by Elizabeth 
# with SQL from the raw HARDI/distributor data.

month_qty_extrap = pd.read_excel('Extrap_byCZ_HARDI_Adjusted_25JAN2022_IsaacCopy.xlsx' , sheet_name = 'Monthly Qty Extrap', usecols='A:E')
ann_qty_extrap   = pd.read_excel('Extrap_byCZ_HARDI_Adjusted_25JAN2022_IsaacCopy.xlsx' , sheet_name = 'Annual Qty Extrap', usecols='A:C')
extrap_table     = pd.read_excel('Extrap_byCZ_HARDI_Adjusted_25JAN2022_IsaacCopy.xlsx' , sheet_name = 'ExtrapStructure', usecols='A:H')
detailed_sales   = pd.read_excel('Extrap_byCZ_HARDI_Adjusted_25JAN2022_IsaacCopy.xlsx' , sheet_name = 'DetailedSales', usecols='A:H')

In [3]:
# Supplier 3 reported partial data for January 2016, and supplier 5 reported partial data for December 2017.

ref_table_month = { 'dictA': {'Supplier': 3, 'Year': 2016, 'Month': 1},
                    'dictB': {'Supplier': 5, 'Year': 2017, 'Month': 12}}

In [4]:
# List of suppliers/years that will need to be extrapolated. Note thate suppliers 9 and 10 are NOT extrapolated despite
# having incomplete data, this is because their total reported sales numbers are not large enough to support extrapolation.

ref_table_ann =    {'row1': {'Supplier': 7, 'Year': 2016, 'Extrapolate?': 1},
                   'row2': {'Supplier': 9, 'Year': 2016, 'Extrapolate?': 0},
                   'row3': {'Supplier': 10, 'Year': 2016, 'Extrapolate?': 0},
                   'row4': {'Supplier': 9, 'Year': 2017, 'Extrapolate?': 0},
                   'row5': {'Supplier': 9, 'Year': 2018, 'Extrapolate?': 0},
                   'row6': {'Supplier': 9, 'Year': 2019, 'Extrapolate?': 0},
                   'row7': {'Supplier': 10, 'Year': 2019, 'Extrapolate?': 0},
                   'row8': {'Supplier': 1, 'Year': 2020, 'Extrapolate?': 1},
                   'row9': {'Supplier': 9, 'Year': 2020, 'Extrapolate?': 0},
                   'row10': {'Supplier': 10, 'Year': 2020, 'Extrapolate?': 0},
                   'row11': {'Supplier': 11, 'Year': 2017, 'Extrapolate?': 1},
                   'row12': {'Supplier': 11, 'Year': 2016, 'Extrapolate?': 1}}

In [5]:
# Add a flag to the monthly extrapolation table for those months indicated in the ref_table_month. Months with flags
# will be extrapolated.

month_qty_extrap['ExtrapMonth?'] = 0

for i in ref_table_month:
    supplier = ref_table_month[i]['Supplier']
    year = ref_table_month[i]['Year']
    month = ref_table_month[i]['Month']
    month_qty_extrap.loc[(month_qty_extrap['Supplier'] == supplier) & (month_qty_extrap['YearSale'] == year) & (month_qty_extrap['MonthSale'] == month), ['Extrapolate']] = 1 

In [6]:
# Used for the monthly extraction, only applied to the two month extrap cases.

def get_monthpct_adjyear(self, supplier, month, yearsale, cz):
    if self['YearSale'] == 2016:
        numer = month_qty_extrap.loc[(self['Supplier'] == supplier) & (self['MonthSale']==month) & (self['CZ']==cz) & (self['YearSale']== yearsale-1),['MonthQuantity']].sum() 
        denom = month_qty_extrap.loc[(self['Supplier'] == supplier) & (self['CZ']==cz) & (self['YearSale']==yearsale-1),['MonthQuantity']].sum()
        month_pct = numer / denom
    else:
        numer = month_qty_extrap.loc[(self['Supplier'] == supplier) & (self['MonthSale']==month) & (self['CZ']==cz) & (self['YearSale']== yearsale+1),['MonthQuantity']].sum() 
        denom = month_qty_extrap.loc[(self['Supplier'] == supplier) & (self['CZ']==cz) & (self['YearSale']==yearsale+1),['MonthQuantity']].sum()
        month_pct = numer / denom
    return float(month_pct)

In [7]:
# Rolls up the monthly quantites to an annual sum.

def get_year_quantity(self, supplier, yearsale, cz):
    total = month_qty_extrap.loc[(self['Supplier'] == supplier) & (self['YearSale']== yearsale) & (self['CZ']== cz),['MonthQuantity']].sum()
    return total 

In [8]:
# Sum of sales for the distributor in the year reported divided by (1 minus % of sales in that month in the adjacent year)

def get_annual_total_extrap(self, supplier, yearsale):
    numer = month_qty_extrap.loc[(self['Supplier'] == supplier) & (self['YearSale']== yearsale),['MonthQuantity']].sum() - self['MonthQuantity']
    denom = 1-self['Monthpct_adj_year']
    total_extrap = numer/denom
    return total_extrap 

In [9]:
# This function calculates the pct share for each supplier to be used for annual extrapolation. For example, 
# supplier 3 is being extrapolated for January 2016. Extrap calc assumes that the pct of sales in january was the same in 
# 2018 as in 2016, this function calcs that pct share. 

def get_2018_share(self, year, supplier, cz):
    if self['YearSale'] == 2019 or self['YearSale'] == 2020:
        numer = ann_qty_extrap.loc[(year==2019) & (self['Supplier'] == supplier) & (self['CZ']== cz),['Total']].sum()
        denom = ann_qty_extrap.loc[(year==2019) & (self['CZ']== cz),['Total']].sum()
        share = numer / denom
    else:
        numer = ann_qty_extrap.loc[(year==2018) & (self['Supplier'] == supplier) & (self['CZ']== cz),['Total']].sum()
        denom = ann_qty_extrap.loc[(year==2018) & (self['CZ']== cz),['Total']].sum()
        share = numer / denom
    return share 

In [10]:
# Calculates the pct share of sales for specific supplier/year/region compared to the TOTAL sales for ALL suppliers
# in that region that year.

def get_annual_share(self, yearsale, cz):
    if self['Total'] == 0:
        share = 0
    else:
        numer = self['Total']
        denom = ann_qty_extrap.loc[(self['YearSale'] == yearsale) & (self['CZ']== cz),['Total']].sum()
        share = numer/denom
    return share 

In [11]:
# Numerator and denominator below used to calculate the total extrapolated quantites for a given year region where extrapolation is required. 

def get_denom(self, year, cz, extrap):
    a = ann_qty_extrap.loc[(self['YearSale'] == year) & (self['CZ']== cz) & (extrap==1),['2018_share']].sum()
    if self['ExtrapAnnual?']== 0:
        c = self['2018_share']
    else:
        c = 0
    denom = 1 - a - c
    return denom

In [12]:
def get_numer(self, year, cz, extrap):
    a = ann_qty_extrap.loc[(self['YearSale'] == year) & (self['CZ']== cz),['Total']].sum()
    b = ann_qty_extrap.loc[(extrap==1) & (self['YearSale'] == year) & (self['CZ']== cz),['Total']].sum()
    if self['ExtrapAnnual?']== 0:
        c = self['Total']
    else:
        c = 0
    numer = a - b - c
    return numer

In [13]:
# Brute force method to assign the appropriate adjacent year of complete data for a given supplier. 
# Extrapolation rests on following assumption: The mix of technologies is, on average, similar for that supplier
# across years. For example, if ductless heat pumps accounted for 5% of a supplier’s total reported sales in 2017,
# then the analysis assumes ductless heat pump sales would similarly account for 5% of this supplier’s total 
# reported sales in 2018. Note that supplier 11 requires extrapolation for both 2016 and 2017 ('112' is used for the time being to 
# differentiate between the two years for supplier 11.)

def get_adjacent_year(self):
    adj = 0 
    if self['Supplier'] == 1:
        adj = 2020
    if self['Supplier'] == 3:
        adj = 2016
    if self['Supplier'] == 5:
        adj = 2017
    if self['Supplier']== 7:
        adj = 2016
    if self['Supplier'] == 11:
        adj = 2017
    if self['Supplier'] == 112:
        adj = 2016
    return adj

In [14]:
def get_extrapolated_unitsA(self, supplier, year, cz, flag):
    a = ann_qty_extrap.loc[(self['Supplier2'] == supplier) & (self['assign_year']== year)  & (self['CZ']== cz) & (1 == flag),['AnnExtrapQty']].sum()
    return a

In [15]:
def get_extrapolated_unitsB(self, supplier, year, cz, flag):
    b = ann_qty_extrap.loc[(self['Supplier2'] == supplier) & (self['assign_year']== year)  & (self['CZ']== cz),['MonthExtrapQty']].sum()
    return b

In [16]:
def get_reported(self, supplier, cz, year, tech, htg, clg, sect):
    reported = detailed_sales.loc[(self['Supplier2'] == supplier) & (self['CZ'] == cz) & (self['assign_year'] == year) & (self['Technology'] == tech) & (self['HeatingEfficiency'] == htg) & (self['CoolingEfficiency'] == clg) & (self['sector'] == sect),['Reported Quantity']].sum()
    return reported

In [17]:
# Adds a flag to the annual extrap table to indicate which suppliers require extrapolation. 

def get_annual_flag(self, supplier, year):
    month_flag = 0
    if self['Supplier'] == 7 and self['YearSale'] == 2016:
        month_flag = 1
    if self['Supplier'] == 1 and self['YearSale'] == 2020:
        month_flag = 1
    if self['Supplier'] == 11 and self['YearSale'] == 2016:
        month_flag = 1
    if self['Supplier'] == 11 and self['YearSale'] == 2017:
        month_flag = 1
    return month_flag

In [18]:
# Calculate the pecect share of sales for a given techo olgy 

def get_pct_share(self, supplier, year, cz):
    numer = self['Quantity']
    denom = extrap_table.loc[(self['Supplier'] == supplier) & (self['yearSale']== year)  & (self['CZ']== cz),['Quantity']].sum()
    return numer/denom

In [19]:
# Add a flag to the annual extrapolation table for those suppliers/years indicated in the ref_table_annual. Supplier/year
# combos with flags will be extrapolated.

ann_qty_extrap['ExtrapAnnual?'] = 0

for i in ref_table_ann:
    supplier = ref_table_ann[i]['Supplier']
    year = ref_table_ann[i]['Year']
    extrap = ref_table_ann[i]['Extrapolate?']
    ann_qty_extrap.loc[(ann_qty_extrap['Supplier'] == supplier) & (ann_qty_extrap['YearSale'] == year) & (extrap == 1), ['ExtrapAnnual?']] = 1 

In [20]:
# For suppliers 3 and 5 the missing month has already been filled in, 
# so extrapolation is simply the total extrpolated units for that supplier/year, 
# multiplied by the pct share for a given technology in the adjacent year.

def get_extrap_final(self, supplier, cz, year,tech, htg, clg, sect):
    if self['Supplier'] == 3 or self['Supplier'] == 5:
        total = self['ExtrapQty']*self['pct_annual']
    else:
        a= self['ExtrapQty']*self['pct_annual']
        b = detailed_sales.loc[(self['Supplier2'] == supplier) & (self['CZ'] == cz) & (self['assign_year'] == year) & (self['Technology'] == tech) & (self['HeatingEfficiency'] == htg) & (self['CoolingEfficiency'] == clg) & (self['sector'] == sect),['Reported Quantity']].sum()
        total = a - b
    return round(total,0)

In [21]:
#Recreating 'Extrapolated' in annual qty etxtrap

def get_extrapolated(self, supplier, year, cz , x):
    numer = month_qty_extrap.loc[(self['Supplier'] == supplier) & (self['YearSale']== year)  & (self['CZ']== cz) & (1 == x),['ExtrapQty']].sum()
    return numer

In [22]:
# Apply functions above to the monthly extrap table. Calculates an extrapolated monthly quantity for the cases 
# where a supplier is missing part of a month of data. Extrapolated quantity equals the percent of sales for that month in the adjacent year of data muliplied by the total 
# extrpolated sales for the given year, minus any reported sales that month.

month_qty_extrap['Monthpct_adj_year'] = month_qty_extrap.apply(get_monthpct_adjyear, args=(month_qty_extrap['Supplier'], month_qty_extrap['MonthSale'], month_qty_extrap['YearSale'] ,month_qty_extrap['CZ']), axis=1)
month_qty_extrap['Annual_Tot_Sales_Extrap'] = month_qty_extrap.apply(get_annual_total_extrap, args=(month_qty_extrap['Supplier'], month_qty_extrap['YearSale']), axis=1)
month_qty_extrap.loc[month_qty_extrap['Extrapolate'] == 1, 'ExtrapQty'] = month_qty_extrap['Annual_Tot_Sales_Extrap']*month_qty_extrap['Monthpct_adj_year'] - month_qty_extrap['MonthQuantity']

In [23]:
# Applies functions above to the annual extap dataframe. Key result here is the 'AnnExtrapQty' column which is the total
# extrapolated sales for a given year/supplier combo. This 'AnnExtrapQty' willl be used later in the extrap table calculations
# to fill in info on specific technologies. 

ann_qty_extrap['YearQuantity']   = ann_qty_extrap.apply(get_year_quantity, args=(month_qty_extrap['Supplier'], month_qty_extrap['YearSale'] ,month_qty_extrap['CZ']), axis=1)
ann_qty_extrap['MonthExtrapQty'] = ann_qty_extrap.apply(get_extrapolated, args=(month_qty_extrap['Supplier'], month_qty_extrap['YearSale'] ,month_qty_extrap['CZ'],month_qty_extrap['Extrapolate']), axis=1)
ann_qty_extrap['Total']          = ann_qty_extrap['YearQuantity'] + ann_qty_extrap['MonthExtrapQty']
ann_qty_extrap['AnnualShare']    = ann_qty_extrap.apply(get_annual_share, args=(ann_qty_extrap['YearSale'], ann_qty_extrap['CZ']), axis=1)
ann_qty_extrap['MonthFlag']      = ann_qty_extrap.apply(get_annual_flag, args=(ann_qty_extrap['Supplier'], ann_qty_extrap['YearSale']), axis=1)
ann_qty_extrap['2018_share']     = ann_qty_extrap.apply(get_2018_share, args=(ann_qty_extrap['YearSale'], ann_qty_extrap['Supplier'], ann_qty_extrap['CZ']), axis=1)
ann_qty_extrap['num'] =          ann_qty_extrap.apply(get_numer, args=(ann_qty_extrap['YearSale'], ann_qty_extrap['CZ'], ann_qty_extrap['ExtrapAnnual?']), axis=1)
ann_qty_extrap['denom']          = ann_qty_extrap.apply(get_denom, args=(ann_qty_extrap['YearSale'], ann_qty_extrap['CZ'], ann_qty_extrap['ExtrapAnnual?']), axis=1)
ann_qty_extrap['AnnExtrapQty']   = ann_qty_extrap['num']/ann_qty_extrap['denom']*ann_qty_extrap['2018_share']
ann_qty_extrap['Extrap - Actual'] = ann_qty_extrap['AnnExtrapQty']-ann_qty_extrap['Total']
ann_qty_extrap = ann_qty_extrap.drop(['num','denom'], axis=1)

In [24]:
# Applies functions above to the extrap_table dataframe. This calulates extrapolated quantities broken down by technology
# and efficiency. The final extrapolated quantity for a given tech/efficiency is equal to the total extrapolated sales for
# that supplier/year muliplied by the share of that given tech/eff in the adjacent complete year.

extrap_table['pct_annual'] = extrap_table.apply(get_pct_share, args=(extrap_table['Supplier'], extrap_table['yearSale'], extrap_table['CZ']), axis=1)
extrap_table['Supplier2'] = extrap_table['Supplier']
extrap_table.loc[extrap_table['Supplier'] == 112, 'Supplier2'] = 11
extrap_table['assign_year'] = extrap_table.apply(get_adjacent_year, axis=1)
extrap_table['ExtrapQty-a'] = extrap_table.apply(get_extrapolated_unitsA, args=(ann_qty_extrap['Supplier'], ann_qty_extrap['YearSale'], ann_qty_extrap['CZ'], ann_qty_extrap['MonthFlag']), axis=1)
extrap_table['ExtrapQty-b'] = extrap_table.apply(get_extrapolated_unitsB, args=(ann_qty_extrap['Supplier'], ann_qty_extrap['YearSale'], ann_qty_extrap['CZ'], ann_qty_extrap['MonthFlag']), axis=1)
extrap_table['ExtrapQty'] = extrap_table['ExtrapQty-a'] + extrap_table['ExtrapQty-b']
extrap_table = extrap_table.drop(['ExtrapQty-a','ExtrapQty-b'], axis=1)

In [25]:
extrap_table['HeatingEfficiency'].fillna(value = 'NA', inplace = True)
extrap_table['CoolingEfficiency'].fillna(value = 'NA', inplace = True)
detailed_sales['HeatingEfficiency'].fillna(value = 'NA', inplace = True)
detailed_sales['CoolingEfficiency'].fillna(value = 'NA', inplace = True)

In [26]:
# Final step of extrapolation. Now all temporal gaps have been filled in and we are ready to aggregate final results.

extrap_table['reported'] = extrap_table.apply(get_reported, args=(detailed_sales['Supplier'], detailed_sales['CZ'], detailed_sales['yearSale'], detailed_sales['Technology'], detailed_sales['HeatingEfficiency'] , detailed_sales['CoolingEfficiency'], detailed_sales['sector']  ), axis=1)
extrap_table['extrapped'] = extrap_table.apply(get_extrap_final, args=(detailed_sales['Supplier'], detailed_sales['CZ'], detailed_sales['yearSale'], detailed_sales['Technology'], detailed_sales['HeatingEfficiency'] , detailed_sales['CoolingEfficiency'], detailed_sales['sector']  ), axis=1)
extrap_table['TOTAL'] = extrap_table['reported'] + extrap_table['extrapped']

In [27]:
# Exlude these technologies for these years due to insufficient reporting. 

extrap_table['filter'] = 'filter_in' 
extrap_table.loc[(extrap_table['Technology'] == 'Heat Pump - Air Source, Variable Capacity') & (extrap_table['assign_year'] == 2016), 'filter'] = "filter_out"
extrap_table.loc[(extrap_table['Technology'] == 'Heat Pump - Packaged Terminal (PTHP)') & (extrap_table['assign_year'] == 2020), 'filter'] = "filter_out"
extrap_table.loc[(extrap_table['Technology'] == 'Variable Refrigerant Flow') & (extrap_table['assign_year'] == 2020), 'filter'] = "filter_out"

In [28]:
# Exlude these rows from final results siince they are being accounted for in the extrap_table.  

detailed_sales['filter'] = 'filter_in' 
detailed_sales.loc[(detailed_sales['Supplier'] == 3) & (detailed_sales['yearSale'] == 2016), 'filter'] = "filter_out"
detailed_sales.loc[(detailed_sales['Supplier'] == 5) & (detailed_sales['yearSale'] == 2017), 'filter'] = "filter_out"
detailed_sales.loc[(detailed_sales['Supplier'] == 7) & (detailed_sales['yearSale'] == 2016), 'filter'] = "filter_out"

In [29]:
# Include these rows in final results since they are not accounted for in the extrap_table.

detailed_sales.loc[(detailed_sales['Supplier'] == 5) & (detailed_sales['yearSale'] == 2017) & (detailed_sales['CZ'] == 'OR/WA')& (detailed_sales['Technology'] == 'Heat Pump - Air Source, Variable Capacity'), 'filter'] = "filter_in"
detailed_sales.loc[(detailed_sales['Supplier'] == 5) & (detailed_sales['yearSale'] == 2017) & (detailed_sales['CZ'] == 'OR/WA')& (detailed_sales['Technology'] == 'Heat Pump - Packaged Terminal (PTHP)'), 'filter'] = "filter_in"
#detailed_sales.loc[(detailed_sales['Supplier'] == 5) & (detailed_sales['yearSale'] == 2017) & (detailed_sales['CZ'] == 'OR/WA')& (detailed_sales['Technology'] == 'Gas Packaged Unit'), 'filter'] = "filter_in"

In [30]:
# Create new dataframes from detailed_sales (no extrap) and extrap_table only including in 'filter in' rows from both.

df1 = detailed_sales[detailed_sales['filter'] == 'filter_in']
df1['extrapped'] = 0
df1['total'] = df1['Reported Quantity']
df1 = df1.drop(['filter'], axis=1)
df2 = extrap_table[extrap_table['filter'] == 'filter_in']


C:\Users\ISAACS~1\AppData\Local\Temp/ipykernel_31156/1165512037.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['extrapped'] = 0
C:\Users\ISAACS~1\AppData\Local\Temp/ipykernel_31156/1165512037.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['total'] = df1['Reported Quantity']


In [31]:
# Select only the neccesary columns to include in the final dataframe.

cols = ['Supplier2','assign_year','CZ','sector','Technology','HeatingEfficiency','CoolingEfficiency','reported','extrapped','TOTAL']
col_names = ['Supplier','yearSale','CZ','Sector','Technology','HeatingEfficiency','CoolingEfficiency','Reported Quantity','extrapped_qty','TOTAL']

In [32]:
df2 = df2[cols]
df2.columns = col_names
df1.columns = col_names

In [33]:
# Create the final dataframe which will be used for final outputs.

all_rows_post = pd.concat([df1, df2], axis=0)

In [34]:
all_rows_post.groupby(['yearSale','Technology'])['TOTAL'].sum().to_csv('d.csv')

In [35]:
all_rows_post.to_pickle("all_rows_post.pkl")